## Reading data 

In [1]:
import sqlite3
import pandas as pd

In [2]:
db =sqlite3.connect('../Data/database.sqlite')
data= pd.read_sql_query(""" SELECT * from Reviews WHERE Score!=3""",db) 

In [3]:
print(data.columns)
print(data.shape)

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
(525814, 10)


In [4]:
data=data.head(1000)
type(data)

pandas.core.frame.DataFrame

In [5]:
score=data['Score']

In [6]:
classified_score=score.map(lambda x:int(x>3))
data.loc['Score'] = classified_score
data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1.0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1.0,1.0,5.0,1.303862e+09,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2.0,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0.0,0.0,1.0,1.346976e+09,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3.0,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1.0,1.0,4.0,1.219018e+09,"""Delight"" says it all",This is a confection that has been around a fe...
3,4.0,B000UA0QIQ,A395BORC6FGVXV,Karl,3.0,3.0,2.0,1.307923e+09,Cough Medicine,If you are looking for the secret ingredient i...
4,5.0,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0.0,0.0,5.0,1.350778e+09,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
'''all the users who have given reviews more than 1 time'''
display0 = pd.read_sql_query("""                   
SELECT UserId, ProfileName, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", db)
display0.head()

,UserId,ProfileName,COUNT(*)
0,#oc-R115TNMSPFT9I7,Breyton,2
1,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",3
2,#oc-R11DNU2NBKQ23Z,Kim Cieszykowski,2
3,#oc-R11O5J5ZVQE25C,Penguin Chick,3
4,#oc-R12KPBODL2B5ZD,Christopher P. Presta,2


In [9]:
'''all the unique users '''
display1 = pd.read_sql_query("""                   
SELECT UserId, ProfileName, COUNT(*)
FROM Reviews
GROUP BY UserId
""", db)
display1.head()

,UserId,ProfileName,COUNT(*)
0,#oc-R103C0QSV1DF5E,C,1
1,#oc-R109MU5OBBZ59U,AayGee,1
2,#oc-R10LFEMQEW6QGZ,Julie,1
3,#oc-R10LT57ZGIB140,dipr,1
4,#oc-R10UA029WVWIUI,Kim D,1


In [10]:
print(str(display0.shape[0]) + " users who gave more than 1 reviews." )
print(str(display1.shape[0])+ " unique users out of " +  str(display1['COUNT(*)'].sum()) + " reviews.")

80668 users who gave more than 1 reviews.
256059 unique users out of 568454 reviews.


# Deduplication

In [11]:
'''duplicate reviews '''

display = pd.read_sql_query(""" 
SELECT UserId, ProfileName,Time,ProductId, COUNT(*)
FROM Reviews
GROUP BY UserId, Time 
HAVING COUNT(*)>2
""", db)
display.head()



,UserId,ProfileName,Time,ProductId,COUNT(*)
0,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",1342396800,B005HG9ESG,3
1,#oc-R11O5J5ZVQE25C,Penguin Chick,1346889600,B005HG9ESG,3
2,#oc-R12MGTQS5KZZRV,"SKY2110 ""SKY2110""",1344211200,B005HG9ESG,3
3,#oc-R13NNUL4EKL4FL,N. Chernyavskaya,1348358400,B005HG9ESG,3
4,#oc-R14ZSRYW2YB41B,A. Crafton,1346284800,B005HG9ESG,3


In [12]:
display.iloc[0]['UserId']

'#oc-R11D9D7SHXIJB9'

In [13]:
duplicate = pd.read_sql_query(""" 
SELECT UserId, ProfileName,Time,ProductId,Text
FROM Reviews
WHERE UserID = "#oc-R11D9D7SHXIJB9"
""", db)
duplicate.head()

,UserId,ProfileName,Time,ProductId,Text
0,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",1342396800,B005HG9ESG,"My wife has recurring extreme muscle spasms, u..."
1,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",1342396800,B005HG9ERW,"My wife has recurring extreme muscle spasms, u..."
2,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",1342396800,B005HG9ET0,"My wife has recurring extreme muscle spasms, u..."


Here, the same user reviews different product Id in same time which is not possible.  While examining the productIds B005HG9ESG, B005HG9ERW, B005HG9ET0 in Amazon, we found out that they referred to same product-Essentia Water.Hence, this is a data error. 

In [14]:
sorted_data=data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort')
sorted_data.shape

(1001, 10)

In [15]:
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(999, 10)

In [16]:
(final['Id'].size*1.0)/(data['Id'].size)*100

99.80019980019979

In [17]:
distorted_data = pd.read_sql_query(""" 
SELECT UserId, ProfileName,Time,ProductId,Text,HelpfulnessNumerator, HelpfulnessDenominator
FROM Reviews
WHERE HelpfulnessNumerator> HelpfulnessDenominator
""", db)
distorted_data

,UserId,ProfileName,Time,ProductId,Text,HelpfulnessNumerator,HelpfulnessDenominator
0,A2V0I904FH7ABY,Ram,1212883200,B001EQ55RW,It was almost a 'love at first bite' - the per...,3,2
1,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",1224892800,B000MIDROQ,My son loves spaghetti so I didn't hesitate or...,3,1


In [18]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [19]:
final.shape

(998, 10)